# Biodata
* Nama: **Virgiawan Teguh Kusuma**
* Email: **181240000833@unisnu.ac.id**
* Domisili: **Kabupaten Jepara, Jawa Tengah**

# Domain Proyek
Smartphone menjadi kebutuhan primer bagi seseorang. Dengan mobilitas yang tinggi, smartphone dapat digunakan untuk menunjang aktivitas sehari-hari seperti bekerja, belajar, bahkan untuk kebutuhan entertain sekali pun.
Data dari E-Journal yang berjudul [Global Smartphones Sales May Have Peaked](https://www.elibrary.imf.org/view/journals/001/2020/070/article-A001-en.xml?ArticleTabs=fulltext) mengungkapkan, jumlah pembelian smartphone yang terjual meningkat pesat dari 2009 hingga 2017. Sementara di tahun-tahun berikutnya tergolong stabil. Mungkin penyebab kenaikan yang drastis tersebut adalah proses transisi penggunaan handphone biasa ke smartphone

Ironisnya, dari data yang saya temukan dari E-Journal yang berjudul [Why on earth did I buy that? A study of regretted appliance purchases](https://royalsocietypublishing.org/doi/10.1098/rsta.2016.0373), smartphone tergolong dalam daftar belanja yang sering disesali, karena kebanyakan orang sering tidak menyesuaikan kebutuhan dan budget mereka sendiri saat membeli smartphone.

Sistem prediksi harga/budget pemilihan smartphone berdasarkan kebutuhan spesifikasi dengan menerapkan teknik machine learning, dapat membantu seseorang dalam menentukan budget yang mereka keluarkan untuk membeli smartphone berdasarkan kebutuhan spesifikasi yang mereka butuhkan.

Ada beberapa faktor yang perlu diperhatikan saat memilih smartphone, diantaranya adalah baterai, memori(RAM), dan Storage(ROM/penyimpanan internal). 
Faktor-faktor tersebut juga sangat mempengaruhi harga pada smartphone.

# Business Understanding
Solusi yang akan saya berikan adalah dengan membuatkan sistem prediksi harga/budget pemilihan smartphone berdasarkan kebutuhan spesifikasi dengan menerapkan teknik machine learning.
Tidak hanya memprediksi Harga smartphone, sistem ini juga memberikan filter pada smartphone yang sudah tidak layak dipakai pada tahun ini agar tetap relevan dengan zaman.

# Data Understanding
Dataset yang digunakan adalah berisi data diambil dari situs online flipkart menggunakan scrapy, serta data ini memiliki basis yang berasal dari India.
Dataset tersebut diambil dari [Android devices and mobiles](https://www.kaggle.com/shreyaspj/android-devices-and-mobiles). Dataset berisi 1 file csv (**mobiles1.csv**) dan beberapa informasi smartphone, yaitu:
* battery: kapasitas baterai
* camera: jumlah kamera dan ukuran lensa
* display: ukuran layar dan tipe panel
* memory: kapasitas RAM, ROM dan Slot memori external
* name: nama Smartphone
* price: harga Smartphone dalam bentuk rupees
* processor: jenis processor yang dipakai berserta kecepatannya
* rating: rating pengguna terhadap smartphone
* reviews: jumlah berapa banyak review pada smartphone
* warranty: garansi pada smartphone

# Load Library

## Load google drive

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Library lainnya

In [80]:
import pandas as pd
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error

# Load Dataset



### Load dataset & Ekstraksi file yang masih berbentuk zip dari GDrive

In [81]:
phone = '/content/drive/MyDrive/archive_7.zip'
phoneRef = zipfile.ZipFile(phone, 'r')
phoneRef.extractall('/content')
phoneRef.close()

### Inisialisasi variable untuk membuat Dataframe

In [82]:
dataframe = pd.read_csv('mobiles1.csv')
dataframe

,battery,camera,display,memory,name,price,processor,rating,reviews,warranty
0,5000 mAh Battery,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",Brand Warranty of 1 Year Available for Mobile ...
1,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",Sunrise Design
2,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",Sunrise Design
3,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",Sunrise Design
4,4000 mAh Battery,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",Dual Nano SIM slots and Memory Card Slot
...,...,...,...,...,...,...,...,...,...,...
979,2000 mAh Battery,5MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) FWVGA Display,1 GB RAM | 8 GB ROM | Expandable Upto 32 MB,"Micromax Bharat 4 (Black, 8 GB)",3590,12 Months Brand Warranty,3.8,105 Reviews,NaN
980,2680 mAh Li-Ion Battery,13MP Rear Camera | 5MP Front Camera,13.21 cm (5.2 inch) Full HD Display,3 GB RAM | 32 GB ROM,"Nextbit Robin (Ember, 32 GB)",19999,Qualcomm Snapdragon 808 MSM8992 Processor,4.0,516 Reviews,Brand Warranty of 1 Year
981,4550 mAh Battery,13MP + 5MP | 20MP Front Camera,15.24 cm (6 inch) Full HD Display,4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,"Gionee A1 Plus (Mocha Gold, 64 GB)",10499,Helio P25 MT 6757CD Processor,4.1,710 Reviews,Brand Warranty of 1 Year Available for Mobile ...
982,2100 mAh Li-Ion Battery,8MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) HD Display,1 GB RAM | 8 GB ROM | Expandable Upto 32 GB,"XOLO Omega 5.0 (Black, 8 GB)",8990,MTK 6592M Processor,3.8,81 Reviews,1 Year Manufacturer Warranty


# Data Preprocessing

## Drop Kolom yang tidak digunakan

In [83]:
dataframe = dataframe.drop(columns=['camera',
                                    'display',
                                    'rating',
                                    'reviews',
                                    'warranty'])
dataframe.head()

,battery,memory,name,price,processor
0,5000 mAh Battery,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor
1,5000 mAh Battery,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor
2,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor
3,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor
4,4000 mAh Battery,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor


## Melihat tipe data tiap kolom

In [84]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   battery    984 non-null    object
 1   memory     984 non-null    object
 2   name       984 non-null    object
 3   price      984 non-null    int64 
 4   processor  983 non-null    object
dtypes: int64(1), object(4)
memory usage: 38.6+ KB


## Split kolom memory dan display
* memisahkan **RAM** dengan ROM pada kolom memory menjadi masing-masing kolom baru.


In [85]:
split_memory = dataframe['memory'].str.split('|', 1, expand=True)

dataframe['RAM(GB)'] = split_memory[0]
dataframe['ROM(GB)'] = split_memory[1]
dataframe.head()

,battery,memory,name,price,processor,RAM(GB),ROM(GB)
0,5000 mAh Battery,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor,4 GB RAM,64 GB ROM | Expandable Upto 512 GB
1,5000 mAh Battery,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor,4 GB RAM,64 GB ROM
2,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4 GB RAM,128 GB ROM
3,5000 mAh Battery,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4 GB RAM,128 GB ROM
4,4000 mAh Battery,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor,3 GB RAM,32 GB ROM | Expandable Upto 256 GB


## Mengubah tipe data
* Mengubah tipe data pada kolom **battery** menjadi *float* dan mengubah judul kolomnya menjadi **battery(mAh)**.
* Mengubah tipe data pada kolom **RAM** dan **ROM** menjadi *integer*.
* Mengubah tipe data pada kolom **price** menjadi *float* dan mengonversinya dari *Rupees* ke *IDR(Rupiah)* dengan mengalikan nilai 1 Rupees dalam IDR.

In [86]:
data_battery= dataframe['battery'].str.split('(\d+)',  expand=True)
data_ram= dataframe['RAM(GB)'].str.split('(\d+)',  expand=True)
data_rom= dataframe['ROM(GB)'].str.split('(\d+)',  expand=True)

dataframe['battery(mAh)'] = pd.to_numeric(data_battery[1], downcast='float')
dataframe['RAM(GB)'] = pd.to_numeric(data_ram[1], downcast='integer')
dataframe['ROM(GB)'] = pd.to_numeric(data_rom[1], downcast='integer')
dataframe['price(IDR)'] = (dataframe['price'] * 191.31 )

## Drop Kolom yang sudah tidak digunakan

In [87]:
dataframe = dataframe.drop(columns=['battery',
                                    'price', 
                                    'name',
                                    'memory',
                                    'processor'])
dataframe

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,4,64,5000.0,1912908.69
1,4,64,5000.0,2104218.69
2,4,128,5000.0,2295528.69
3,4,128,5000.0,2295528.69
4,3,32,4000.0,1434633.69
...,...,...,...,...
979,1,8,2000.0,686802.90
980,3,32,2680.0,3826008.69
981,4,64,4550.0,2008563.69
982,1,8,2100.0,1719876.90


## Cek tipe data

In [88]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RAM(GB)       984 non-null    int16  
 1   ROM(GB)       984 non-null    int16  
 2   battery(mAh)  984 non-null    float32
 3   price(IDR)    984 non-null    float64
dtypes: float32(1), float64(1), int16(2)
memory usage: 15.5 KB


# Data Preparation

## Filtering Smartphone
Menetapkan syarat minimal spesifikasi handphone di tahun 2021, yaitu:
* Minimal kapasitas baterai 3000 mAh
* RAM 3GB
* ROM/Penyimpanan Internal 32GB

In [89]:
loc_data=dataframe.loc[
                       (dataframe['battery(mAh)'] >= 3000) &
                       (dataframe['RAM(GB)'] >= 3) &
                       (dataframe['ROM(GB)'] >= 32)
                       ]

## Inisialisai variable baru hasil Filtering

In [90]:
newDataframe = loc_data

## Cek Data

In [91]:
newDataframe.head()

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,4,64,5000.0,1912908.69
1,4,64,5000.0,2104218.69
2,4,128,5000.0,2295528.69
3,4,128,5000.0,2295528.69
4,3,32,4000.0,1434633.69


## Cek Tipe Data

In [92]:
newDataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731 entries, 0 to 983
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RAM(GB)       731 non-null    int16  
 1   ROM(GB)       731 non-null    int16  
 2   battery(mAh)  731 non-null    float32
 3   price(IDR)    731 non-null    float64
dtypes: float32(1), float64(1), int16(2)
memory usage: 17.1 KB


## Cek data ada berapa baris dan kolom

In [93]:
newDataframe.shape

(731, 4)

## Cek statistik description dari data (mean, kuartil, standard deviation, dll.)

In [94]:
newDataframe.describe()

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
count,731.000000,731.000000,731.000000,7.310000e+02
mean,5.149111,95.124487,4144.493652,3.567210e+06
std,2.003677,66.297954,670.548889,2.556974e+06
min,3.000000,32.000000,3000.000000,9.546369e+05
25%,4.000000,64.000000,3782.500000,2.007703e+06
50%,4.000000,64.000000,4000.000000,2.869459e+06
75%,6.000000,128.000000,4500.000000,4.017319e+06
max,12.000000,512.000000,6000.000000,2.008736e+07


## Cek missing value

In [95]:
newDataframe.isnull().sum()

RAM(GB)         0
ROM(GB)         0
battery(mAh)    0
price(IDR)      0
dtype: int64

In [96]:
newDataframe

,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,4,64,5000.0,1912908.69
1,4,64,5000.0,2104218.69
2,4,128,5000.0,2295528.69
3,4,128,5000.0,2295528.69
4,3,32,4000.0,1434633.69
...,...,...,...,...
974,6,64,4000.0,4015596.90
975,6,64,4000.0,4780836.90
976,6,64,5000.0,2678148.69
981,4,64,4550.0,2008563.69


## Inisialisasi variabel x dan y untuk nanti split data

In [97]:
x = newDataframe.drop(columns='price(IDR)')
y = newDataframe['price(IDR)']

## Split data menjadi train & test dengan perbandingan 80:20.


In [98]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

# Model Development
Di tahap modeling, saya menggunakan 3 model regresi yaitu **Linear Regression, Random Forest, dan Gradient Booster**. Lalu performa tiap-tiap model dibandingkan dan dapat ditarik kesimpulan bahwa model dari Random Forest lah yang memiliki tingkat error paling kecil berdasarkan Mean Squared Error (MSE) dan Mean Absolute Error (MAE).

## Linear Regression


In [99]:
lr = LinearRegression()
lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)

lr_df = pd.DataFrame({'y_true': y_test,
                          'y_predict': lr_predict})

In [100]:
# Mencari nilai koefisien dan intercept.
print(lr.coef_)
print(lr.intercept_)

[ 6.953526e+05  9.682177e+03 -2.382959e+02]
67089.25


In [101]:
lr.score(x_test, y_test)

0.5900831226897991

## Random Forest

In [102]:
rf_model = RandomForestRegressor(n_estimators=50, max_depth=16,
                                 random_state=55, n_jobs=-1)
rf_model.fit(x_train, y_train)
rf_predict = rf_model.predict(x_test)

rf_df = pd.DataFrame({'y_true': y_test,
                          'y_predict': rf_predict})

In [103]:
rf_model.score(x_test, y_test)

0.5110395237065304

## Gradient Booster

In [104]:
xgb_model = XGBRegressor()
xgb_model.fit(x_train, y_train)
xgb_predict = xgb_model.predict(x_test)

xgb_df = pd.DataFrame({'y_true': y_test,
                          'y_predict': xgb_predict})

[19:29:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [105]:
xgb_model.score(x_test, y_test)

0.49256230061594386

# Model Evaluation
Berdasarkan hasil metrik evaluasi yang digunakan pada kasus regresi di proyek ini adalah sebagai berikut:
* **Mean Squared Error (MSE)**, merepresentasikan rata-rata perbedaan kuadrat antara nilai aktual dan prediksi pada dataset. MSE mengukur varians dari residual. MSE sangat baik dalam memberikan gambaran terhadap seberapa konsisten model yang dibangun karena model dengan varian kecil dapat memberi hasil yang relatif konsisten.  
![Rumus MSE](https://www.gstatic.com/education/formulas2/397133473/en/mean_squared_error.svg)

* **Mean Absolute Error (MAE)**, merepresentasikan rata-rata perbedaan mutlak antara nilai aktual dan prediksi pada dataset. MAE memberikan rata-rata error dari keseluruhan data.  
![Rumus MAE](https://gisgeography.com/wp-content/uploads/2014/08/mae-formula.png)

## Linear Regression

In [106]:
lr_mse = mean_squared_error(y_test, lr_predict)
lr_mae = mean_absolute_error(y_test, lr_predict)

lr_df.head()

,y_true,y_predict
295,9565308.69,9865286.00
45,3347733.69,4406192.00
36,2102496.90,1581327.00
901,2869458.69,2634123.25
224,4399938.69,7155364.00


## Random Forest

In [107]:
rf_mse = mean_squared_error(y_test, rf_predict)
rf_mae = mean_absolute_error(y_test, rf_predict)
rf_df.head()

,y_true,y_predict
295,9565308.69,1.087097e+07
45,3347733.69,4.351822e+06
36,2102496.90,2.154782e+06
901,2869458.69,2.549039e+06
224,4399938.69,5.711560e+06


## Gradient Boost

In [108]:
xgb_mse = mean_squared_error(y_test, xgb_predict)
xgb_mae = mean_absolute_error(y_test, xgb_predict)

xgb_df.head()

,y_true,y_predict
295,9565308.69,9047948.00
45,3347733.69,4283885.00
36,2102496.90,2165378.50
901,2869458.69,2455154.75
224,4399938.69,5935549.00


## Membandingkan Ke-3 Nya

In [109]:
metrics_df = pd.DataFrame({'MSE': [lr_mse, rf_mse, xgb_mse],
                           'MAE': [lr_mae, rf_mae, xgb_mae]},
                          index=['Linear Regression', 'Random Forest', 'Gradient Booster'])
metrics_df

,MSE,MAE
Linear Regression,2.030860e+12,955508.772891
Random Forest,2.422468e+12,844544.065076
Gradient Booster,2.514010e+12,896406.491429


# Mulai memprediksi harga smartphone berdasarkan spesifikasi:
<!-- * nilai pertama adalah **RAM**
* nilai kedua adalah **ROM/Peyimpanan Internal**
* nilai ketiga adalah **battery** -->

In [110]:
ram = int(input('masukkan kapasitas RAM yang diinginkan (tulis angkanya saja) : '))
rom = int(input('masukkan kapasitas ROM yang diinginkan (tulis angkanya saja) : '))
battery = float(input('masukkan kapasitas Baterai yang diinginkan (tulis angkanya saja) : '))

masukkan kapasitas RAM yang diinginkan (tulis angkanya saja) : 4
masukkan kapasitas ROM yang diinginkan (tulis angkanya saja) : 64
masukkan kapasitas Baterai yang diinginkan (tulis angkanya saja) : 4000


In [111]:
print(type(ram))
print(type(rom))
print(type(battery))

<class 'int'>
<class 'int'>
<class 'float'>


In [112]:
price = rf_model.predict([[ram,rom,battery]])

In [113]:
# Konversi hasil prediksi dari array ke desimal
price = np.round([price], decimals=1)

In [114]:
# Print perkiraan harga smartphone sesuai spesifikasi yang diinginkan
print('Perkiraan Harga Smartphone sesuai dengan spesifikasi yang Anda inginkan adalah Rp.',price[0][0])

Perkiraan Harga Smartphone sesuai dengan spesifikasi yang Anda inginkan adalah Rp. 2414545.3
